In [1]:
import pandas as pd
import time

import json
import httplib2
import sys
import codecs
       
import re

import numpy as np
import math
import ast

import warnings
warnings.filterwarnings("ignore")

import json
#!pip install geog
import geog
import shapely.geometry
from shapely.geometry import Point

In [2]:
df = pd.read_csv('new_venues_clean.csv', encoding = "ISO-8859-1") 

In [15]:
### Input
x_0 = 34.05223
y_0 = -118.24368
x_f = 37.77493
y_f = -122.41942

print ("Origin Lat/Long", x_0, y_0 )
print ("Destination Lat/Long", x_f, y_f )
#approximate distance between points (aka "radius")
r_a = 30
print ("Radius", r_a )


Origin Lat/Long 34.05223 -118.24368
Destination Lat/Long 37.77493 -122.41942
Radius 30


In [16]:
### Input - Origin Lat/Long, Destination Lat/Long, Radius and Dataframe
def poi_inside_box(x_0,y_0,x_f,y_f,r_a,df):
    d_t_1 = ((y_f-y_0)**2+(x_f-x_0)**2)**.5
    print ('Euclidean Distance between Origin and Destination', d_t_1)
    
    #Haversine distance from origin to Destination
    dlat = math.radians(x_f-x_0)
    dlon = math.radians(y_f-y_0)
    radius = 6371 # km
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(x_f)) \
        * math.cos(math.radians(x_0)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d_t = radius * c
    print ('Haversine Distance between Origin and Destination', d_t)
    
    #actual number of points needed
    n = round(d_t/r_a)-1
    print ('Number of points', n)
    
    x_delta = (x_f - x_0)/n
    y_delta = (y_f - y_0)/n

    #in future iterations this could be points along the route
    list_of_centroids = [(x_0+i*x_delta, y_0+i*y_delta) for i in range(1,n)]
    print ('List of Centroids\n', list_of_centroids)
        
    #list of tuples in the form of (x_min, x_max, y_min, y_max)
    d = r_a * 1000  # meters
    polygon_points = 4
    angles = np.linspace(0, 360, polygon_points)
         
    df['in_the_box'] = False
    df['centroid_index'] = 99
    for i,row in df.iterrows():
        for (x,y) in list_of_centroids:
            #print ("Index of Centroid", centroid_list.index((x,y)))
            p = shapely.geometry.Point([y, x])
            polygon = geog.propagate(p, angles, d)
            box = shapely.geometry.Polygon(polygon)
            point = Point(df['lng'][i], df['lat'][i])
            if point.within(box):
                df['in_the_box'][i] = point.within(box)
                df['centroid_index'][i] = list_of_centroids.index((x,y)) 
                break
    return df

In [17]:
df1 = poi_inside_box(x_0,y_0,x_f,y_f,r_a,df)

Euclidean Distance between Origin and Destination 5.59422021711695
Haversine Distance between Origin and Destination 559.1229009538062
Number of points 18
List of Centroids
 [(34.25904666666667, -118.47566555555555), (34.46586333333333, -118.7076511111111), (34.67268, -118.93963666666667), (34.87949666666667, -119.17162222222223), (35.08631333333334, -119.40360777777778), (35.29313, -119.63559333333333), (35.499946666666666, -119.86757888888889), (35.706763333333335, -120.09956444444444), (35.913579999999996, -120.33155), (36.120396666666664, -120.56353555555556), (36.32721333333333, -120.79552111111111), (36.53403, -121.02750666666667), (36.74084666666666, -121.25949222222222), (36.94766333333333, -121.49147777777777), (37.15448, -121.72346333333334), (37.36129666666667, -121.9554488888889), (37.56811333333333, -122.18743444444445)]


In [18]:
df1[df1['in_the_box'] == True]

,Unnamed: 0,name,id,category,address,lat,lng,city,state,price_tier,serves,bucket,big_bucket,like_counts,rating,image_url,in_the_box,centroid_index
71,71,Duke's Malibu,3fd66200f964a520e1ef1ee3,'seafood restaurant',21150 Pacific Coast Hwy,34.036429,-118.635453,Malibu,CA,3,"Happy Hour, Brunch & more",3.0,1,459,8.3,https://fastly.4sqi.net/img/general/300x500/33...,True,0
72,72,The Country Kitchen,4a5be975f964a520fbbb1fe3,'burger joint',21239 Pacific Coast Hwy,34.037293,-118.637700,Malibu,CA,2,Brunch & Lunch,0.0,1,32,8.3,https://fastly.4sqi.net/img/general/300x500/46...,True,0
76,76,Moonshadows,43655e80f964a52067291fe3,'new american restaurant',20356 Pacific Coast Hwy,34.037553,-118.618413,Malibu,CA,3,"Happy Hour, Brunch & more",3.0,1,335,8.8,https://fastly.4sqi.net/img/general/300x500/73...,True,0
83,83,Cafe Mimosa,4b3a488ff964a520bd6325e3,'café',395 S Topanga Canyon Blvd,34.084113,-118.599840,Topanga,CA,1,Breakfast,1.0,1,18,7.9,https://fastly.4sqi.net/img/general/300x500/16...,True,0
85,85,The Canyon Bistro & Wine Bar in Topanga,4b134bd4f964a520379623e3,'french restaurant',120 N Topanga Canyon Blvd,34.089965,-118.603967,Topanga,CA,2,"Dinner, Lunch & more",7.0,1,23,7.9,https://fastly.4sqi.net/img/general/300x500/fN...,True,0
89,89,Rocco's in the Canyon,4b80993cf964a520fd7f30e3,'italian restaurant',123 Topanga Canyon Blvd,34.088971,-118.604200,Topanga,CA,2,Dessert,7.0,1,7,7.1,https://fastly.4sqi.net/img/general/300x500/4L...,True,0
92,92,Water Lily Cafe,4b7de9bbf964a520a1d92fe3,'sandwich place',106 N Topanga Canyon Blvd,34.089516,-118.603603,Topanga,CA,1,NaN,0.0,1,5,6.4,https://fastly.4sqi.net/img/general/300x500/17...,True,0
101,101,Fernwood Market,4b78a304f964a5205cdb2ee3,'deli / bodega',446 S Topanga Canyon Blvd,34.080992,-118.600801,Topanga,CA,1,NaN,0.0,1,0,7.2,https://fastly.4sqi.net/img/general/300x500/11...,True,0
154,154,Palapas Restaurant & Cantina,4b5bb372f964a520e51029e3,'mexican restaurant',21 Seascape Village,36.952236,-121.877093,Aptos,CA,3,"Dinner, Lunch & more",4.0,1,21,7.2,https://fastly.4sqi.net/img/general/300x500/D7...,True,14
157,157,Full of Beans,4b3e4760f964a520929a25e3,'café',10 Seascape Vlg,36.950513,-121.874381,Aptos,CA,1,NaN,1.0,1,4,7.0,https://fastly.4sqi.net/img/general/300x500/80...,True,14
